In [1]:
import assessmentModule as amod
import glob
import matplotlib.pyplot as plt
import numpy as np
import os
import scipy.signal as ss
%matplotlib inline
"""
Script for reading Matlab files, filtering SmO2, tHb,
and HR and fitting a polynomial to early-time data.

"""

__author__ = 'Todd Minehardt'


def getValuesDict(in_dict, key_type, tags, start_idx, end_idx):
    """Return dict keys on tags."""
    tmp_dict = {}
    for key in in_dict[key_type].keys():
        if key in tags:
            values = in_dict[key_type][key][start_idx:end_idx]
            tmp_dict[key + '_values'] = values
    return tmp_dict


# Key type. Should be one of 'assessment', 'sweep', 'process',
# 'stat', 'device', 'warmup'.
key_type = 'warmup'

# Suffix for Matlab files. Should correspond to key.
matfile_suffix = '_' + key_type + '.mat'

# Define working directory where Matlab files are and output directory.
data_dir = '/Users/todd/data/COMPLETE/warmup'
out_dir = '/Users/todd/data/COMPLETE/warmup/SmO2'

# List process files, get assessment IDs from them.
os.chdir(data_dir)
matlab_files = glob.glob('*' + matfile_suffix)
assessment_ids = [i.split('_', 1)[0] for i in matlab_files]

# Get rid of assessments on the exclude_file.
exclude_file = '/Users/todd/code/AnalysisTools/config/exclude_assessments.txt'
with open(exclude_file, 'r') as fn:
    exclude_list = []
    for line in fn:
        exclude_list.append(line)
exclude_list = [i.strip('\n') for i in exclude_list]

# Amend bad_list if assessment ID appears on exclude list.
excluded_list = []
for assess_id in assessment_ids:
    if assess_id in exclude_list:
        assessment_ids.remove(assess_id)
        excluded_list.append(assess_id)
        
# Get bad plots list.
bad_plots_file = '/Users/todd/data/COMPLETE/warmup/SmO2/bad_plots_list.txt'
with open(bad_plots_file, 'r') as fn:
    bad_plots_list = []
    for line in fn:
        bad_plots_list.append(line)
bad_plots_list = [i.strip('\n') for i in bad_plots_list]

for assess_id in assessment_ids:
    if assess_id in bad_plots_list:
        assessment_ids.remove(assess_id)
        excluded_list.append(assess_id)
        
# Build a dict keyed on assessment ID, containing all information
# from relevant Matlab files. This returns a dict of dicts.
data = {}
for assess_id in assessment_ids:
    data[assess_id] = amod.getMatlabData(assess_id + matfile_suffix)

# Plot parameters.
plt.rcParams['figure.figsize'] = 10, 8
plt.rcParams['legend.loc'] = 'best'

# Tags are keys of interest.
tags = ['HR', 'SmO2', 'tHb']

# Define start and end indices for data arrays.
start_idx = 161
end_idx = None

# Exclude assessments where > 10% of values are NaN and
# where > 10% of HR values are 1.0.
assess_to_del_list = []
bad_HR_list = []
missing_list = []
for assess_id in data.keys():
    values_dict = getValuesDict(data[assess_id],
                                key_type,
                                tags,
                                start_idx,
                                end_idx)

    # Check for NaN.
    HR_vals = values_dict['HR_values']
    SmO2_vals = values_dict['SmO2_values']
    tHb_vals = values_dict['tHb_values']
    hr_nan_pct = np.isnan(HR_vals).sum() / len(HR_vals) * 100.0
    smo2_nan_pct = np.isnan(SmO2_vals).sum() / len(SmO2_vals) * 100.0
    thb_nan_pct = np.isnan(tHb_vals).sum() / len(tHb_vals) * 100.0
    if (hr_nan_pct > 10.0 and smo2_nan_pct > 10.0 and
        thb_nan_pct > 10.0):
        missing_list.append(assess_id)
        assess_to_del_list.append(assess_id)

    # Check for HR > 20.
    hr_gt_cnt = np.count_nonzero(np.where(HR_vals > 20.0))
    hr_pct = hr_gt_cnt / len(HR_vals) * 100.0
    if hr_pct < 90.0:
        bad_HR_list.append(assess_id)
        assess_to_del_list.append(assess_id)

# Remove data dict entries that are not good.
assess_to_del_list = list(set(assess_to_del_list))
for entry in assess_to_del_list:
    del data[entry]

In [4]:
# Plot remaining assessments.
good_list = []
for assess_id in data.keys():    
    SmO2 = data[assess_id][key_type]['SmO2'][start_idx:end_idx]
    # Filter SmO2 if Stage 1 to Stage 2 transition is > 3%.
    smo2_st1_mean = SmO2[600:900].mean()
    smo2_st2_mean = SmO2[900:1200].mean()
    delta_smo2 = (smo2_st1_mean - smo2_st2_mean) * 100.0
    if delta_smo2 > 3.0:
        x2 = np.arange(0, len(SmO2)) / 5
        plt.plot(x2, SmO2, 'r-', label='SmO2')
        plt.xlabel('time (s)')
        plt.ylabel('SmO2 (%)', color='red')
        plt.axvline(180, color='green')
        plt.axvspan(150, 210, color='green', alpha=0.25)
        plt.legend()        
        plt.title('SmO2 Change at Stage 1 to Stage 2 for LT Assessments')
        out_file = os.path.join(out_dir, assess_id + '.png')
        plt.savefig(out_file)
        plt.close()
        good_list.append(assess_id)
        
good_list.sort()
excluded_list.sort()
missing_list.sort()
bad_HR_list.sort()
with open(os.path.join(out_dir, 'good_LT_SmO2_warmup.txt'), 'a') as fn:
    for line in good_list:
        fn.write(line + '\n')

with open(os.path.join(out_dir, 'excluded_LT_SmO2_warmup.txt'), 'a') as fn:
    for line in excluded_list:
        fn.write(line + '\n')

with open(os.path.join(out_dir, 'missing_LT_SmO2_warmup.txt'), 'a') as fn:
    for line in missing_list:
        fn.write(line + '\n')

with open(os.path.join(out_dir, 'bad_HR_LT_SmO2_warmup.txt'), 'a') as fn:
    for line in bad_HR_list:
        fn.write(line + '\n')